In [ ]:
%matplotlib inline 

import numpy as np
import pandas as pd
pd.options.display.float_format = '{:,.4f}'.format
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.font_manager import FontProperties

sns.set_style('darkgrid')
titleFont = FontProperties(weight='bold', size=20)
axisFont = FontProperties(weight='bold', size=14) 

from trading.exchange import BackTestExchange
from trading.dataManager import CurrencyDataManager
from trading.money.contract import ContractPair, Contract
from trading.statistics import Statistics
from trading.money.transaction import BuyTransaction, SellTransaction
from calculator.talibWrapper import calculateTalib

In [ ]:
from calculator.talibWrapper import talibFunctionInfo
talibFunctionInfo('BBANDS')

# Load Data

In [ ]:
dataRoot = 'D:\\Dropbox\\My work\\krypl-project\\dataLabeled\\poloniex\\5min'
fileName = 'USDT_BTC_5min_2016-01-01_2017-12-31.tsv'
file = '{dataRoot}\\{fileName}'.format(dataRoot=dataRoot, fileName=fileName)

usdtBtc = pd.read_csv(file, sep='\t').sort_values('timestamp').reset_index().drop('index', axis=1)

trainRatio = 0.7
trainSize = int(usdtBtc.shape[0] * trainRatio)
usdtBtcTrain = usdtBtc.iloc[:trainSize]
usdtBtcTest = usdtBtc.iloc[trainSize:]

usdtBtcManagerTrain = CurrencyDataManager(usdtBtcTrain)
usdtBtcManagerTest = CurrencyDataManager(usdtBtcTest)
wallet = {'usdt': 1000}
contractPair = ContractPair('usdt', 'btc')

# Bollinger Bands

Bollinger Bands consist of a moving average and two standard deviations,
one above the moving average and one below. The important thing
to know about Bollinger Bands is that they contain up to 95% of the
closing prices, depending on the settings.

- A buy signal is generated when prices move below the lower Bollinger Band.
- A sell signal is generated when prices move above the upper Bollinger Band.

Source: Markus Heitkoetter: The Complet Guide to Day Trading

In [ ]:
data = {
            'close': usdtBtc.head(10)['close'].values,
        }
        
upper, middle, lower = calculateTalib('BBANDS', data, {'timeperiod': 5})

In [ ]:
from copy import deepcopy

class BollingerBandsStrategy:
    def __init__(self, exchange, dataManager, historyLen, contractPair, tradeSize, willingLoss, opened=False):
        self.exchange = exchange
        self.walletStart = deepcopy(self.exchange.wallet)
        self.dataManager = dataManager
        self.historyLen = historyLen
        self.contractPair = contractPair
        self.opened = opened
        self.tradeSize = tradeSize
        self.willingLoss = willingLoss

    def calculateBBands(self, history, col, timePeriod):
        upper, middle, lower = calculateTalib('BBANDS', {col: history[col].values}, {'timeperiod': timePeriod})
        return upper[-1], middle[-1], lower[-1]
    
    
    def isRisky(self, priceBought, actualPrice):
        return self.currentReturn(priceBought, actualPrice) < -self.willingLoss 
    
    
    def currentReturn(self, priceBought, actualPrice):
        return float(actualPrice - priceBought) / priceBought
    
    
    def trade(self):
        
        wasBellow = False
        while self.dataManager.hasTick():            
            history, price = self.dataManager.tick(self.historyLen)
            if history.shape[0] < self.historyLen:
                continue

            upper, middle, lower = self.calculateBBands(history, 'close', self.historyLen)
            
            lastLow = history['low'].iloc[-1]
            if not self.opened and wasBellow and  lastLow > lower:
                print(self.dataManager.time, 'openning trade: ', price, end='\t')
                amount = self.tradeSize / price
                self.exchange.buy(self.contractPair, amount, price)
                priceBought = price
                self.opened = True
                wasBellow = False
            elif self.opened and self.isRisky(priceBought, price):
                print(self.dataManager.time, 'stopLoss: ', price, self.currentReturn(priceBought, price) * 100)
                self.opened = False
                amount = self.exchange.balance(self.contractPair.tradeContract)
                self.exchange.sell(self.contractPair, amount, price)
            elif self.opened and price > upper:
                print(self.dataManager.time, 'profit: ', price, self.currentReturn(priceBought, price) * 100)
                self.opened = False
                amount = self.exchange.balance(self.contractPair.tradeContract)
                self.exchange.sell(self.contractPair, amount, price)

                
            wasBellow = price < lower
                
    def stats(self, contractName):
        return Statistics(contractName, self.walletStart[contractName]).evaluate(self.exchange.transactions)

# Learning

In [ ]:
dataManager = deepcopy(usdtBtcManagerTrain)
exchange = BackTestExchange(dataManager, deepcopy(wallet), 0.0025)
strategy = BollingerBandsStrategy(exchange, dataManager, 20, contractPair, 100, 0.05)
strategy.trade()
strategy.stats('usdt').report()

In [ ]:
for t in exchange.transactions:
    print(t)

In [ ]:
allStats

In [ ]:
import time

allStats = None
for slow in range(96, 96*30, 96):
    print('fast:', slow, end='')
    for fast in range(96, slow, 96):
        print('\t slow', fast, end='')
        start_time = time.time()
        dataManager = deepcopy(usdtBtcManager)
        exchange = BackTestExchange(dataManager, deepcopy(wallet), 0.0035)
        strategy = CrossoverMAStrategy(exchange, dataManager, slow, fast, contractPair)
        strategy.trade()
        stats = strategy.stats('usdt').report().transpose().reset_index().rename(columns={'index': 'contract'})
        stats['timePeriodFast'] = fast
        stats['timePeriodSlow'] = slow
        if allStats is None:
            allStats = stats
        else:
            allStats = allStats.append(stats)
        print("\t %.2f seconds" % (time.time() - start_time))
        print("\t", end='')
    print()

# Look on transactions

In [ ]:
from matplotlib.finance import candlestick_ohlc
import matplotlib.dates as mdates

import time
import datetime as dt

def timestampToDate(timestamp):
    return dt.datetime.fromtimestamp(int(timestamp))

def strTimeToTimestamp(strTime):
    return dt.datetime.strptime(strTime, "%Y-%m-%d %H:%M:%S").timestamp()


def calculateMA(history, col, timePeriod):
        return calculateTalib('MA', {col: history[col].values}, {'timeperiod': timePeriod})


def plotCandles(df, currencyPair):
    DATA = df[['open', 'high', 'low', 'close', 'volume', 'date', 'timestamp']].copy()
    DATA = DATA.reset_index()
    DATA["dateStr"] = DATA["date"]
    DATA["date"]  = DATA["timestamp"].apply(timestampToDate).apply(mdates.date2num)

    plt.figure(figsize=(16, 8))
    ax = plt.gca()
    candlestick_ohlc(ax, DATA.values, width=.6, colorup='#53c156', colordown='#ff1717')

    ax.xaxis_date()
    
    xDates = DATA['dateStr']
    step = int(xDates.shape[0] / 20) + 1
    xDates[[i for i in range(xDates.shape[0]) if i % step != 0]] = ''
    
    plt.xticks(range(DATA.shape[0]), DATA['dateStr'], rotation=45)
    plt.title(currencyPair, fontproperties=titleFont)
    

def plotMA(df, col, timeperiod):
    ma = calculateMA(df, col, timeperiod)
    return plt.plot(ma)


def calculateBB(history, col, timePeriod):
        upper, middle, lower = calculateTalib('BBANDS', {col: history[col].values}, {'timeperiod': timePeriod})
        return upper, middle, lower


def plotBB(df, col, timeperiod):
    upper, middle, lower = calculateBB(df, col, timeperiod)
    return plt.plot(upper), plt.plot(middle), plt.plot(lower)


def plotTransaction(transaction, i):
    color = 'k' if type(transaction) == BuyTransaction else 'm'
    plt.scatter(i, transaction.price.value, s=200, c=color)

    
def plotTransactionWithSurroundings(prices, timePeriod, transaction):
    i = transaction.timestamp
    start = i - 40 if i - 20 > 0 else 0
    end = i + 20 if i + 20 < prices.shape[0] else prices.shape[0]-1
    selected = prices.ix[start:end, :].reset_index()
    plotCandles(selected, '')
    plotTransaction(transaction, 39)
    l1, l2, l3 = plotBB(selected, 'close', timePeriod)
    plt.legend(l1+l2+l3, ['Upper', 'Mean', 'Lower'])

In [ ]:
len(strategy.exchange.transactions)

In [ ]:
t = 2002
transaction = strategy.exchange.transactions[t]
plotTransactionWithSurroundings(usdtBtc, 20, transaction)
transaction = strategy.exchange.transactions[t+1]
plotTransactionWithSurroundings(usdtBtc, 20, transaction)

In [ ]:
transaction

# Evaluate Statistics Crossover MA

In [ ]:
allStats.to_csv('results/crossoverMA-results.tsv', sep='\t', index=False)

In [ ]:
allStats.head()

In [ ]:
def plotResults(stats, xCol):
    yCols=['numberOfTrades', 'totalProfit', 'avgProfit', 'winPercentage', 'avgWinTrade', 
           'avgLossTrade', 'profitFactor', 'maxDrawdown']
    
    f, axarr = plt.subplots(3, 3, figsize=[16,10])
    for i in range(axarr.shape[0]):
        for j in range(axarr.shape[1]):
            step = i * axarr.shape[0] + j
            if step >= len(yCols):
                break
            col = yCols[step]
            ax = axarr[i, j]
            ax.scatter(stats['timePeriodSlow'], stats['timePeriodFast'], s=stats[col], alpha=0.5)
            ax.set_title(col)
#             ax.set_xlabel('timePeriodSlow')
            ax.set_ylabel('timePeriodFast')

In [ ]:
plotResults(allStats, 'timePeriodSlow')

In [ ]:
allStats.query('totalProfit > 200').sort_values('numberOfTrades', ascending=False)

# Conclusion

Strategie Moving Average crossover, uz dokaze na paru usdt-btc, vytvorit az 25% zisk za necele dva roky. Bohuzel vytvari celkem 
malo obchodu (za dva roky kolem 50-60). Ze zkusennosti z marketu, a sledovani volatility moc dobre vime, ze lze vydelat daleko vice. Crossover moving average by vsak mohl byt dobrou featurou, pro machine learning techniky.